In [ ]:
import numpy as np
from scipy.ndimage import distance_transform_edt
from skimage.morphology import skeletonize
from skan import Skeleton, summarize
import skan.csr
import tifffile as tiff

# Assuming `binary_volume` is the binary image of the vascular structure
binary_volume = tiff.imread('outputs/output_skeleton_volume.tif')

# Step 1: Create the distance map
distance_map = distance_transform_edt(binary_volume)

# Step 2: Skeletonize the volume
skeleton_volume = skeletonize(binary_volume)

# Step 3: Multiply the distance map by the skeleton to get the width at each skeleton point
skeleton_widths = distance_map * skeleton_volume

# Step 4: Create the skeleton object in skan using the width data
skeleton = Skeleton(skeleton_volume, skeleton_image=skeleton_widths)

# Step 5: Calculate mean, max, and median widths for each path
# Summarize provides the essential metrics by default
summary = summarize(skeleton)

# Add columns for mean, max, and median widths

# Mean width per path
summary['mean_width'] = [np.mean(skeleton.path_with_data(i)[1]) for i in range(skeleton.n_paths)]

# Max width per path using reduceat
summary['max_width'] = [np.maximum.reduceat(skeleton.path_with_data(i)[1], range(len(skeleton.path_with_data(i)[1]))) for i in range(skeleton.n_paths)]

# Median width per path
summary['median_width'] = [np.median(skeleton.path_with_data(i)[1]) for i in range(skeleton.n_paths)]

# `summary` now includes columns for mean, max, and median width for each branch in the skeleton
print(summary)
